# 1 Setup


## 1.1 Import Libraries

In [ ]:
!pip install earthengine-api
!pip install geemap
!pip install Pillow
!pip install tensorflow

ERROR: Operation cancelled by user
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 7.8 MB/s eta 0:00:00


In [ ]:
import ee
import geemap
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
from sklearn.neighbors import LocalOutlierFactor
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from google.colab import files
from google.colab import drive
import random
from PIL import Image, ImageDraw, ImageFont
import time
import pandas as pd
import sys
import json
from datetime import datetime
import re
from typing import List, Dict, Set, Tuple
import gc
from tqdm import tqdm

In [ ]:
ee.Authenticate()
ee.Initialize(project='germanytiles')
drive.mount('/content/drive')

Mounted at /content/drive


## 1.2 Grid & Map Creation

* ~Bremen~
* ~Hamburg~
* ~Berlin~
* ~Saarland~
* ~Schleswig-Holstein~
* ~Thüringen~
* ~Sachsen~
* ~Rheinland-Pfalz~
* ~Sachsen-Anhalt~
* ~Hessen~
* ~Mecklenburg-Vorpommern~
* ~Brandenburg~
* ~Nordrhein-Westfalen~
* ~Baden-Württemberg~
* ~Niedersachsen~
* Bayern

In [ ]:
germany = ee.FeatureCollection("FAO/GAUL/2015/level1") \
            .filter(ee.Filter.eq('ADM0_NAME', 'Germany')) #\
            #.filter(ee.Filter.eq('ADM1_NAME', 'Bayern'))

germany = germany.filterBounds(germany).geometry()

grid = germany.coveringGrid('EPSG:32632', 2000)
grid = grid.filterBounds(germany)

# Get grid size
#grid_size = grid.size().getInfo()
#print(f"Total grid cells to process: {grid_size}")

In [ ]:
image_collection = ee.ImageCollection("COPERNICUS/S2_HARMONIZED")
germany_buffered_x = germany.buffer(5000)

# filter for only pictures after second filling and cloud coverage & study area
image_collection = image_collection \
    .filterDate("2021-02-01", "2023-08-01") \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 5)) \
    .filter(ee.Filter.bounds(germany)) \
    .filter(ee.Filter.bounds(germany_buffered_x))

# Mosaic the image collection
mosaic_image = image_collection.mosaic()

# Print the number of images (to double check if there's enough)
print(f"Number of Images in the Collection: {image_collection.size().getInfo()}")

Number of Images in the Collection: 3426


In [ ]:
Map = geemap.Map(center=[51, 10], zoom=6)
Map.addLayer(germany, {}, "Germany")
Map.addLayer(
    mosaic_image.clip(germany_buffered_x),
    {'bands': ['B4', 'B3', 'B2'],
     'min': 0.0,
     'max': 2000},
    'real_colours',
    True,
    1
)
Map.addLayer(grid, {"color": "red"}, "Grid sample (limit 500)")
Map

Map(center=[51, 10], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(chil…

# 2 Image Creation & Export



### 2.1 Functions

In [ ]:
# Function to add coordinate-based IDs
def add_bbox_coordinate_id(feature):
    bounds = feature.geometry().bounds(maxError=1)
    coords = bounds.coordinates().get(0)
    sw_corner = ee.List(coords).get(0)
    lon = ee.Number(ee.List(sw_corner).get(0)).format('%.3f')
    lat = ee.Number(ee.List(sw_corner).get(1)).format('%.3f')
    custom_id = ee.String(lon).cat('_').cat(lat)
    return feature.set('custom_id', custom_id)

# Export parameters
viz_params = {
    'bands': ['B4', 'B3', 'B2'],
    'min': 0.0,
    'max': 2000,
}

folder = "Germany_Tiles"

**WATCH THE FORCE FRESH START**

In [ ]:
# BATCH EXPORT CONFIGURATION
BATCH_SIZE = 20  # Number of tiles to export in each batch
WAIT_BETWEEN_BATCHES = 30  # Seconds to wait between batches

# FORCE FRESH START - Set to True to ignore existing progress and start over
FORCE_FRESH_START = False  # Change to False to resume from existing progress

# PROGRESS TRACKING SYSTEM - SAVED TO GOOGLE DRIVE
PROGRESS_FILE = "/content/drive/MyDrive/germany_export_progress.json"  # Saved to Drive

In [ ]:
def load_progress():
    """Load progress from Google Drive"""
    # Check if we should force a fresh start
    if FORCE_FRESH_START:
        print("FORCE_FRESH_START is True - ignoring existing progress and starting fresh")
        return {
            'completed': [],
            'failed': [],
            'last_processed_index': -1,
            'total_cells': grid_size,
            'start_time': datetime.now().isoformat(),
            'active_tasks': []  # Track currently running tasks
        }

    if os.path.exists(PROGRESS_FILE):
        try:
            with open(PROGRESS_FILE, 'r') as f:
                data = json.load(f)
                print(f"Loaded progress file from Drive: {len(data.get('completed', []))} completed exports")
                return data
        except Exception as e:
            print(f"Error loading progress file: {e}")
            print("Starting fresh...")
    else:
        print("No progress file found on Drive - starting fresh")

    return {
        'completed': [],
        'failed': [],
        'last_processed_index': -1,
        'total_cells': grid_size,
        'start_time': datetime.now().isoformat(),
        'active_tasks': []  # Track currently running tasks
    }

def save_progress(progress_data):
    """Save progress to Google Drive"""
    try:
        # Ensure the directory exists
        os.makedirs(os.path.dirname(PROGRESS_FILE), exist_ok=True)

        progress_data['last_updated'] = datetime.now().isoformat()
        with open(PROGRESS_FILE, 'w') as f:
            json.dump(progress_data, f, indent=2)
        print(f"  → Progress saved to Drive: {len(progress_data['completed'])} completed, {len(progress_data['failed'])} failed")
    except Exception as e:
        print(f"  → Warning: Could not save progress to Drive: {e}")

        # Fallback: save locally in Colab
        try:
            local_file = "germany_export_progress_local.json"
            with open(local_file, 'w') as f:
                json.dump(progress_data, f, indent=2)
            print(f"  → Fallback: Progress saved locally as {local_file}")
        except:
            print("  → Could not save progress anywhere!")

def create_batch_tasks(start_index, batch_size, progress_data):
    """Create a batch of export tasks - returns list of (task, custom_id, grid_index)"""
    tasks = []

    end_index = min(start_index + batch_size, grid_size)

    try:
        # Instead of accessing one by one, get a chunk of grid features
        print(f"    → Getting grid features {start_index} to {end_index-1}...")
        grid_chunk = grid.toList(end_index - start_index, start_index)
        chunk_size = grid_chunk.size().getInfo()

        if chunk_size == 0:
            print(f"    ⚠ No features found in range {start_index}-{end_index}")
            return tasks

        print(f"    → Successfully got {chunk_size} features")

        # Process each feature in the chunk
        for local_idx in range(chunk_size):
            global_idx = start_index + local_idx

            try:
                print(f"    → Preparing cell {global_idx} for batch export...")

                # Get feature from the chunk
                feat = ee.Feature(grid_chunk.get(local_idx))
                feat_with_id = add_bbox_coordinate_id(feat)

                custom_id = feat_with_id.get('custom_id').getInfo()

                # Check if already completed
                if custom_id in progress_data['completed']:
                    print(f"    ⚠ SKIP: {custom_id} already exported")
                    continue

                # Check if previously failed
                if custom_id in [item['custom_id'] for item in progress_data['failed']]:
                    print(f"    ⚠ RETRY: {custom_id} previously failed, retrying...")

                geom = feat.geometry()

                # Check geometry validity
                area = geom.area(maxError=1).getInfo()
                if area <= 0:
                    error_msg = f"Invalid geometry at index {global_idx} - area is {area}"
                    print(f"    ✗ ERROR: {error_msg}")
                    failure_record = {
                        'custom_id': custom_id,
                        'grid_index': global_idx,
                        'error': error_msg,
                        'timestamp': datetime.now().isoformat()
                    }
                    progress_data['failed'].append(failure_record)
                    continue

                task = ee.batch.Export.image.toDrive(
                    image=mosaic_image.clip(geom).visualize(**viz_params),
                    description=f"tile_{custom_id}",
                    folder=folder,
                    fileNamePrefix=f"tile_{custom_id}",
                    region=geom,
                    crs='EPSG:32632',
                    scale=10,
                    maxPixels=1e9,
                    fileFormat='GEO_TIFF'
                )

                tasks.append((task, custom_id, global_idx))
                print(f"    ✓ Prepared task for {custom_id} (area: {area:.0f} m²)")

            except Exception as e:
                error_msg = str(e)
                print(f"    ✗ ERROR preparing cell {global_idx}: {error_msg}")

                failure_record = {
                    'custom_id': f"unknown_at_index_{global_idx}",
                    'grid_index': global_idx,
                    'error': f"Failed during task preparation: {error_msg}",
                    'timestamp': datetime.now().isoformat(),
                    'batch_start': start_index,
                    'grid_size_at_time': grid_size
                }
                progress_data['failed'].append(failure_record)

    except Exception as e:
        error_msg = str(e)
        print(f"    ✗ ERROR getting grid chunk {start_index}-{end_index}: {error_msg}")

        # Mark entire batch as failed
        for i in range(start_index, end_index):
            failure_record = {
                'custom_id': f"chunk_failed_at_index_{i}",
                'grid_index': i,
                'error': f"Failed to get grid chunk: {error_msg}",
                'timestamp': datetime.now().isoformat()
            }
            progress_data['failed'].append(failure_record)

    return tasks

def start_batch_tasks(tasks):
    """Start all tasks in a batch"""
    started_tasks = []

    for task, custom_id, grid_index in tasks:
        try:
            print(f"    → Starting export for {custom_id}...")
            task.start()
            started_tasks.append((task, custom_id, grid_index))
            print(f"    ✓ Started: {custom_id}")

        except Exception as e:
            print(f"    ✗ Failed to start {custom_id}: {str(e)}")
            # This will be handled as a failure in the calling function

    return started_tasks

def check_batch_completion(started_tasks, progress_data, timeout_minutes=30):
    """Monitor batch completion and update progress"""
    print(f"  → Monitoring {len(started_tasks)} active tasks...")

    completed_in_batch = []
    failed_in_batch = []
    still_running = list(started_tasks)

    start_time = time.time()
    timeout_seconds = timeout_minutes * 60

    while still_running and (time.time() - start_time) < timeout_seconds:
        new_still_running = []

        for task, custom_id, grid_index in still_running:
            try:
                status = task.status()
                state = status['state']

                if state == 'COMPLETED':
                    print(f"    ✓ COMPLETED: {custom_id}")
                    completed_in_batch.append(custom_id)
                    progress_data['completed'].append(custom_id)

                elif state in ['FAILED', 'CANCEL_REQUESTED', 'CANCELLED']:
                    error_msg = status.get('error_message', f'Task {state}')
                    print(f"    ✗ FAILED: {custom_id} - {error_msg}")

                    failure_record = {
                        'custom_id': custom_id,
                        'grid_index': grid_index,
                        'error': error_msg,
                        'timestamp': datetime.now().isoformat()
                    }
                    progress_data['failed'].append(failure_record)
                    failed_in_batch.append(custom_id)

                else:  # READY, RUNNING, etc.
                    new_still_running.append((task, custom_id, grid_index))

            except Exception as e:
                print(f"    ⚠ Error checking status for {custom_id}: {str(e)}")
                new_still_running.append((task, custom_id, grid_index))

        still_running = new_still_running

        if still_running:
            print(f"    → {len(still_running)} tasks still running, checking again in 30 seconds...")
            time.sleep(30)

    # Handle timeout case
    if still_running:
        print(f"  ⚠ TIMEOUT: {len(still_running)} tasks still running after {timeout_minutes} minutes")
        print("  → These tasks may complete later. Check your Google Drive and Earth Engine task manager.")
        for task, custom_id, grid_index in still_running:
            print(f"    - {custom_id} (still running)")

    return completed_in_batch, failed_in_batch

### 2.2 Batch Export

In [ ]:
# ROBUST BATCH EXPORT PROCESS
print("\n" + "="*70)
print("STARTING ROBUST BATCH EXPORT PROCESS")
print("="*70)

# Load existing progress
progress_data = load_progress()

# Determine where to start
START_FROM_INDEX = max(0, progress_data.get('last_processed_index', -1) + 1)

print(f"Grid size: {grid_size}")
print(f"Batch size: {BATCH_SIZE}")
print(f"Wait between batches: {WAIT_BETWEEN_BATCHES} seconds")
print(f"Previously completed: {len(progress_data['completed'])}")
print(f"Previously failed: {len(progress_data['failed'])}")
print(f"Starting from cell index: {START_FROM_INDEX}")
print(f"Remaining cells to process: {max(0, grid_size - START_FROM_INDEX)}")

if len(progress_data['completed']) > 0:
    print(f"Progress file found - resuming from where we left off!")
    print(f"Last few completed: {progress_data['completed'][-3:]}")

# Show failed attempts if any
if len(progress_data['failed']) > 0:
    print(f"\nPreviously failed cells:")
    for failed_item in progress_data['failed'][-5:]:  # Show last 5 failures
        print(f"  - {failed_item['custom_id']}: {failed_item['error']}")

print(f"\nProgress file: {PROGRESS_FILE}")
print("Starting in 5 seconds... (Ctrl+C to cancel)")
time.sleep(5)

print("\n" + "-"*70)
print("BATCH EXPORT PROCESS STARTED")
print("-"*70)

total_batches = (grid_size - START_FROM_INDEX + BATCH_SIZE - 1) // BATCH_SIZE
current_batch = 0

for batch_start in range(START_FROM_INDEX, grid_size, BATCH_SIZE):
    current_batch += 1
    batch_end = min(batch_start + BATCH_SIZE, grid_size)

    print(f"\n{'='*50}")
    print(f"BATCH {current_batch}/{total_batches}")
    print(f"Processing cells {batch_start} to {batch_end-1}")
    print(f"{'='*50}")

    # Create batch tasks
    print(f"[Step 1/4] Creating batch tasks...")
    batch_tasks = create_batch_tasks(batch_start, BATCH_SIZE, progress_data)

    if not batch_tasks:
        print("  → No tasks to process in this batch (all already completed or failed to prepare)")
        progress_data['last_processed_index'] = batch_end - 1
        save_progress(progress_data)
        continue

    print(f"  → Created {len(batch_tasks)} tasks for this batch")

    # Start all tasks in the batch
    print(f"[Step 2/4] Starting {len(batch_tasks)} export tasks...")
    started_tasks = start_batch_tasks(batch_tasks)

    if not started_tasks:
        print("  → No tasks successfully started in this batch")
        progress_data['last_processed_index'] = batch_end - 1
        save_progress(progress_data)
        continue

    print(f"  → Successfully started {len(started_tasks)} tasks")

    # Monitor completion
    print(f"[Step 3/4] Monitoring task completion...")
    completed_batch, failed_batch = check_batch_completion(started_tasks, progress_data)

    # Update progress
    print(f"[Step 4/4] Updating progress...")
    progress_data['last_processed_index'] = batch_end - 1
    save_progress(progress_data)

    # Batch summary
    print(f"\n  BATCH {current_batch} SUMMARY:")
    print(f"    Completed: {len(completed_batch)}")
    print(f"    Failed: {len(failed_batch)}")
    print(f"    Total completed so far: {len(progress_data['completed'])}")
    print(f"    Total failed so far: {len(progress_data['failed'])}")

    # Wait between batches (except for last batch)
    if batch_start + BATCH_SIZE < grid_size:
        print(f"\n  → Waiting {WAIT_BETWEEN_BATCHES} seconds before next batch...")
        time.sleep(WAIT_BETWEEN_BATCHES)

# Final summary
print(f"\n{'='*70}")
print("BATCH EXPORT PROCESS COMPLETED!")
print(f"{'='*70}")
print(f"Total cells in grid: {grid_size}")
print(f"Total completed exports: {len(progress_data['completed'])}")
print(f"Total failed exports: {len(progress_data['failed'])}")
print(f"Export folder: {folder}")
print(f"Progress file location: {PROGRESS_FILE} (Google Drive)")

if len(progress_data['failed']) > 0:
    print(f"\nNote: {len(progress_data['failed'])} cells have failed at some point.")
    print("You may want to investigate these failures manually.")
    print("Failed exports:")
    for failed_item in progress_data['failed'][-10:]:  # Show last 10 failures
        print(f"  - {failed_item['custom_id']}: {failed_item['error']}")

print("Make sure to check your Google Drive and Earth Engine Task Manager!")
print(f"{'='*70}")

# Save final progress
save_progress(progress_data)

Streaming output truncated to the last 5000 lines.

BATCH 106/178
Processing cells 16840 to 16859
[Step 1/4] Creating batch tasks...
    → Getting grid features 16840 to 16859...
    → Successfully got 9 features
    → Preparing cell 16840 for batch export...
    ✓ Prepared task for 11.406_50.113 (area: 3986600 m²)
    → Preparing cell 16841 for batch export...
    ✓ Prepared task for 11.434_50.112 (area: 3986532 m²)
    → Preparing cell 16842 for batch export...
    ✓ Prepared task for 11.462_50.112 (area: 3986464 m²)
    → Preparing cell 16843 for batch export...
    ✓ Prepared task for 11.490_50.111 (area: 3986394 m²)
    → Preparing cell 16844 for batch export...
    ✓ Prepared task for 11.518_50.110 (area: 3986324 m²)
    → Preparing cell 16845 for batch export...
    ✓ Prepared task for 11.545_50.110 (area: 3986254 m²)
    → Preparing cell 16846 for batch export...
    ✓ Prepared task for 11.573_50.109 (area: 3986182 m²)
    → Preparing cell 16847 for batch export...
    ✓ Prepar

# 3 Load and Preprocess Images

## 3.1 Counting and checking Number of Images


In [ ]:
def count_files_in_folder(folder_path):
    """Counts the number of files in a given folder path."""
    if not os.path.exists(folder_path):
        print(f"Folder not found: {folder_path}")
        return -1
    try:
        # List all entries in the folder and filter for files
        entries = os.listdir(folder_path)
        files = [entry for entry in entries if os.path.isfile(os.path.join(folder_path, entry))]
        return len(files)
    except Exception as e:
        print(f"An error occurred while counting files: {e}")
        return -1

In [ ]:
# Specify the folder path in your Google Drive
folder_to_count = '/content/drive/MyDrive/Tiles/Tiles_all'

# Count the files
file_count = count_files_in_folder(folder_to_count)

if file_count != -1:
    print(f"The number of files in '{folder_to_count}' is: {file_count}")

## 3.2 Process & Move Images

In [ ]:
class ImageProcessor:
    """High-performance image processor with batch operations and threading."""

    def __init__(self, source_folder, destination_folder, max_workers=4, batch_size=100):
        self.source_folder = source_folder
        self.destination_folder = destination_folder
        self.max_workers = max_workers
        self.batch_size = batch_size
        self.lock = Lock()
        self.stats = {
            'tif_converted': 0,
            'png_moved': 0,
            'errors': 0,
            'skipped': 0
        }

    def process_single_file(self, filename):
        """Process a single file (convert TIF or move PNG)."""
        try:
            file_path = os.path.join(self.source_folder, filename)

            # Skip directories
            if os.path.isdir(file_path):
                with self.lock:
                    self.stats['skipped'] += 1
                return f"⚠ Skipped directory: {filename}"

            # Process .tif files (convert to PNG)
            if filename.lower().endswith(('.tif', '.tiff')):
                return self._process_tif_file(filename, file_path)

            # Process existing .png files (just move them)
            elif filename.lower().endswith('.png'):
                return self._process_png_file(filename, file_path)

            else:
                with self.lock:
                    self.stats['skipped'] += 1
                return f"⚠ Skipped non-image: {filename}"

        except Exception as e:
            with self.lock:
                self.stats['errors'] += 1
            return f"✗ Error processing {filename}: {str(e)[:100]}..."

    def _process_tif_file(self, filename, file_path):
        """Convert TIF to PNG and move to destination."""
        try:
            # Generate PNG filename
            png_filename = os.path.splitext(filename)[0] + '.png'
            final_png_path = os.path.join(self.destination_folder, png_filename)

            # Handle naming conflicts
            final_png_path = self._get_unique_filename(final_png_path)

            # Open, convert, and save directly to destination
            with Image.open(file_path) as img:
                # Convert mode if necessary (optimized)
                if img.mode not in ('RGB', 'L'):
                    if img.mode in ('RGBA', 'LA'):
                        # Use white background for transparency
                        background = Image.new('RGB', img.size, (255, 255, 255))
                        if img.mode == 'RGBA':
                            background.paste(img, mask=img.split()[-1])
                        else:
                            background.paste(img, mask=img.split()[-1])
                        img = background
                    else:
                        img = img.convert('RGB')

                # Save with optimized settings for speed
                img.save(final_png_path, format='PNG', optimize=False, compress_level=1)

            # Remove original TIF file
            os.remove(file_path)

            with self.lock:
                self.stats['tif_converted'] += 1

            return f"✓ Converted: {filename} -> {os.path.basename(final_png_path)}"

        except Exception as e:
            with self.lock:
                self.stats['errors'] += 1
            return f"✗ TIF conversion failed {filename}: {str(e)[:100]}..."

    def _process_png_file(self, filename, file_path):
        """Move existing PNG file to destination."""
        try:
            final_png_path = os.path.join(self.destination_folder, filename)

            # Handle naming conflicts
            final_png_path = self._get_unique_filename(final_png_path)

            # Move the file
            shutil.move(file_path, final_png_path)

            with self.lock:
                self.stats['png_moved'] += 1

            return f"✓ Moved: {filename} -> {os.path.basename(final_png_path)}"

        except Exception as e:
            with self.lock:
                self.stats['errors'] += 1
            return f"✗ PNG move failed {filename}: {str(e)[:100]}..."

    def _get_unique_filename(self, filepath):
        """Generate unique filename if conflict exists."""
        if not os.path.exists(filepath):
            return filepath

        directory = os.path.dirname(filepath)
        base_name = os.path.splitext(os.path.basename(filepath))[0]
        extension = os.path.splitext(filepath)[1]

        counter = 1
        while os.path.exists(filepath):
            new_filename = f"{base_name}_{counter}{extension}"
            filepath = os.path.join(directory, new_filename)
            counter += 1

        return filepath

    def process_batch(self, file_batch, batch_num, total_batches):
        """Process a batch of files with threading."""
        print(f"🚀 Processing batch {batch_num}/{total_batches} ({len(file_batch)} files)")

        with concurrent.futures.ThreadPoolExecutor(max_workers=self.max_workers) as executor:
            # Submit all files in batch
            future_to_file = {executor.submit(self.process_single_file, filename): filename
                             for filename in file_batch}

            # Collect results (optional: reduce verbosity)
            batch_results = []
            for future in concurrent.futures.as_completed(future_to_file):
                result = future.result()
                batch_results.append(result)

        # Print sample results from batch (not all to reduce spam)
        if batch_num % 10 == 1 or batch_num == total_batches:  # Show every 10th batch + last
            print(f"  Sample from batch {batch_num}: {batch_results[0] if batch_results else 'No results'}")

        # Force garbage collection to free memory
        gc.collect()

        return batch_results

def convert_and_organize_images_fast():
    """
    High-performance converter for large numbers of images.
    Uses threading and batch processing for optimal speed.
    """

    # Configuration
    source_folder = '/content/drive/MyDrive/Germany_Tiles'
    destination_folder = '/content/drive/MyDrive/Tiles/Tiles_all'

    # Performance settings - adjust based on your needs
    max_workers = 6  # Number of threads (increase for more speed, decrease if hitting quotas)
    batch_size = 150  # Files per batch (larger = more memory, smaller = more overhead)

    print(f"🚀 HIGH-PERFORMANCE IMAGE PROCESSOR")
    print(f"   Threads: {max_workers}")
    print(f"   Batch size: {batch_size}")
    print(f"   Source: {source_folder}")
    print(f"   Destination: {destination_folder}")
    print("-" * 60)

    # Initialize processor
    processor = ImageProcessor(source_folder, destination_folder, max_workers, batch_size)

    # Create destination folder
    try:
        os.makedirs(destination_folder, exist_ok=True)
        print(f"✓ Destination folder ready: {destination_folder}")
    except Exception as e:
        print(f"✗ Error creating destination folder: {e}")
        return False

    # Verify source folder
    if not os.path.exists(source_folder):
        print(f"✗ Source folder does not exist: {source_folder}")
        return False

    # Get all files
    try:
        os.chdir(source_folder)
        all_files = [f for f in os.listdir() if os.path.isfile(f)]
        print(f"✓ Found {len(all_files)} files to process")
    except Exception as e:
        print(f"✗ Error reading directory: {e}")
        return False

    # Filter relevant files
    image_files = [f for f in all_files if f.lower().endswith(('.tif', '.tiff', '.png'))]
    print(f"✓ Found {len(image_files)} image files (TIF/PNG)")

    if not image_files:
        print("⚠ No image files found to process")
        return True

    # Process in batches
    start_time = time.time()
    total_batches = (len(image_files) + batch_size - 1) // batch_size

    print(f"🔄 Processing {len(image_files)} files in {total_batches} batches...")

    for i in range(0, len(image_files), batch_size):
        batch_num = (i // batch_size) + 1
        file_batch = image_files[i:i + batch_size]

        batch_start = time.time()
        processor.process_batch(file_batch, batch_num, total_batches)
        batch_time = time.time() - batch_start

        # Progress update
        files_processed = min(i + batch_size, len(image_files))
        progress = (files_processed / len(image_files)) * 100
        avg_time_per_file = batch_time / len(file_batch)

        print(f"📊 Progress: {progress:.1f}% ({files_processed}/{len(image_files)}) "
              f"| Batch time: {batch_time:.1f}s | Avg: {avg_time_per_file:.3f}s/file")

    # Final summary
    total_time = time.time() - start_time
    print("\n" + "="*60)
    print("🎯 FINAL SUMMARY:")
    print(f"✓ TIF files converted: {processor.stats['tif_converted']}")
    print(f"✓ PNG files moved: {processor.stats['png_moved']}")
    print(f"⚠ Files skipped: {processor.stats['skipped']}")
    print(f"✗ Errors: {processor.stats['errors']}")
    print(f"⏱ Total time: {total_time:.1f} seconds")
    print(f"🚀 Average speed: {len(image_files)/total_time:.1f} files/second")
    print(f"📁 Destination: {destination_folder}")
    print("="*60)

    return processor.stats['errors'] == 0


In [ ]:
# Run the optimized function
if __name__ == "__main__":
    success = convert_and_organize_images_fast()
    if success:
        print("\n🎉 All files processed successfully!")
    else:
        print("\n⚠ Some files had errors. Check the log above.")

## 3.3 Missing Images **[idea abandoned]**

### 3.3.1 Double Check for missing images

In [ ]:
def add_bbox_coordinate_id(feature):
    bounds = feature.geometry().bounds(maxError=1)
    coords = bounds.coordinates().get(0)
    sw_corner = ee.List(coords).get(0)
    lon = ee.Number(ee.List(sw_corner).get(0)).format('%.3f')
    lat = ee.Number(ee.List(sw_corner).get(1)).format('%.3f')
    custom_id = ee.String(lon).cat('_').cat(lat)
    return feature.set('custom_id', custom_id)

# Your existing code
germany = ee.FeatureCollection("FAO/GAUL/2015/level1") \
            .filter(ee.Filter.eq('ADM0_NAME', 'Germany')) \
            .filter(ee.Filter.eq('ADM1_NAME', 'Bayern'))

germany = germany.filterBounds(germany).geometry()

grid = germany.coveringGrid('EPSG:32632', 2000)
grid = grid.filterBounds(germany)

# Add custom IDs to each grid cell
grid_with_ids = grid.map(add_bbox_coordinate_id)



In [ ]:
# Get as a simple list
custom_ids_list = grid_with_ids.aggregate_array('custom_id').getInfo()
print(f"Total grid cells: {len(custom_ids_list)}")
print(f"First few IDs: {custom_ids_list[:5]}")


Total grid cells: 18299
First few IDs: ['10.163_47.271', '10.190_47.271', '10.216_47.271', '10.243_47.270', '10.164_47.289']


In [ ]:
missing_tiles_to_process = []

In [ ]:
# Clean up the entries list - remove "tile_" prefix and ".png" suffix
cleaned_entries = []
for entry in entries:
    # Remove "tile_" from the beginning and ".png" from the end
    cleaned_entry = entry.replace("tile_", "").replace(".png", "")
    cleaned_entries.append(cleaned_entry)

In [ ]:
# Convert to sets for efficient comparison
grid_ids_set = set(custom_ids_list)
entries_set = set(cleaned_entries)

# Find IDs that are in the grid but NOT in entries (missing tiles)
missing_tiles = grid_ids_set - entries_set

# Convert back to list if needed
missing_tiles_list = list(missing_tiles)

print(f"Total grid cells: {len(custom_ids_list)}")
print(f"Total entries (after cleaning): {len(cleaned_entries)}")
print(f"Missing tiles: {len(missing_tiles_list)}")

# Show some examples
print(f"\nFirst few missing tiles: {missing_tiles_list[:10]}")

# Optional: Also find tiles that are in entries but not in grid (shouldn't happen normally)
extra_tiles = entries_set - grid_ids_set
if extra_tiles:
    print(f"\nExtra tiles in entries (not in grid): {len(extra_tiles)}")
    print(f"Examples: {list(extra_tiles)[:5]}")


# Append missing tiles to a list
missing_tiles_to_process.extend(missing_tiles_list)


print(f"\nMissing tiles added to list: {len(missing_tiles_to_process)}")
print(f"First few tiles to process: {missing_tiles_to_process[:5]}")


Total grid cells: 18299
Total entries (after cleaning): 70377
Missing tiles: 7569

First few missing tiles: ['13.219_49.089', '11.352_47.829', '11.555_50.289', '9.782_50.045', '11.823_49.599', '10.233_49.088', '10.877_48.737', '12.228_49.445', '10.469_48.725', '11.818_48.538']

Extra tiles in entries (not in grid): 59647
Examples: ['12.121_54.107', '13.846_52.879', '9.909_47.850', '8.464_52.978', '14.351_51.904']

Missing tiles added to list: 22822
First few tiles to process: ['13.296_52.433', '13.149_52.438', '13.177_52.419', '13.327_52.450', '13.650_52.437']


In [ ]:
# export missing_tiles_to_process as a csv file to my drive

missing_tiles_df = pd.DataFrame(missing_tiles_to_process, columns=['custom_id'])

# Specify the path where you want to save the CSV in your Drive
csv_path = '/content/drive/MyDrive/missing_tiles_to_process.csv'

# Export the DataFrame to a CSV file in your Drive
missing_tiles_df.to_csv(csv_path, index=False)

print(f"Successfully exported missing tiles to {csv_path}")

Successfully exported missing tiles to /content/drive/MyDrive/missing_tiles_to_process.csv


### 3.3.2 missing ids create and export

In [ ]:
# Your existing setup (make sure these are defined)
germany = ee.FeatureCollection("FAO/GAUL/2015/level1") \
             .filter(ee.Filter.eq('ADM0_NAME', 'Germany')) \

germany = germany.filterBounds(germany).geometry()

image_collection = ee.ImageCollection("COPERNICUS/S2_HARMONIZED")
germany_buffered_x = germany.buffer(5000)

image_collection = image_collection \
     .filterDate("2021-02-01", "2023-08-01") \
     .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 5)) \
     .filter(ee.Filter.bounds(germany)) \
     .filter(ee.Filter.bounds(germany_buffered_x))

mosaic_image = image_collection.mosaic()

In [ ]:
# Read the CSV file with missing custom IDs
csv_path = '/content/drive/MyDrive/missing_tiles_to_process.csv'
missing_df = pd.read_csv(csv_path)

In [ ]:
# Extract custom IDs from the CSV (assuming they're in a column called 'Filename' or 'custom_id')
# Adjust the column name based on your CSV structure
if 'Filename' in missing_df.columns:
    # Extract custom_id from filename (assuming format like "tile_12.345_67.890.png")
    missing_custom_ids = missing_df['Filename'].str.replace('tile_', '').str.replace('.png', '').str.replace('.tif', '').tolist()
elif 'custom_id' in missing_df.columns:
    missing_custom_ids = missing_df['custom_id'].tolist()
else:
    # If the structure is different, adjust accordingly
    print("Available columns:", missing_df.columns.tolist())
    print("Please update the code to match your CSV structure")
    missing_custom_ids = []

print(f"Found {len(missing_custom_ids)} missing custom IDs to export")
print(f"First few IDs: {missing_custom_ids[:5]}")


# Function to create geometry from custom_id
def create_geometry_from_custom_id(custom_id, cell_size=2000):
    """Create geometry from custom_id (lon_lat format)"""
    try:
        # Parse custom_id (format: "12.345_67.890")
        lon_str, lat_str = custom_id.split('_')
        lon = float(lon_str)
        lat = float(lat_str)

        # Create southwest corner point in WGS84
        sw_point = ee.Geometry.Point([lon, lat])

        # Project to UTM zone 32N to work in meters
        utm_sw = sw_point.transform('EPSG:32632', maxError=1)
        utm_coords = utm_sw.coordinates()

        # Get UTM coordinates
        utm_x = ee.Number(utm_coords.get(0))
        utm_y = ee.Number(utm_coords.get(1))

        # Create the four corners of the rectangle in UTM
        sw_utm = [utm_x, utm_y]
        se_utm = [utm_x.add(cell_size), utm_y]
        ne_utm = [utm_x.add(cell_size), utm_y.add(cell_size)]
        nw_utm = [utm_x, utm_y.add(cell_size)]

        # Create polygon from corners in UTM
        utm_polygon = ee.Geometry.Polygon([sw_utm, se_utm, ne_utm, nw_utm], proj='EPSG:32632')

        # Transform back to WGS84
        wgs84_geometry = utm_polygon.transform('EPSG:4326', maxError=1)

        return wgs84_geometry

    except Exception as e:
        print(f"Error creating geometry for {custom_id}: {str(e)}")
        return None

# Alternative simpler function if UTM approach still has issues
def create_geometry_from_custom_id_simple(custom_id, cell_size_degrees=0.018):
    """Create geometry from custom_id using simple degree offset (backup method)"""
    try:
        # Parse custom_id (format: "12.345_67.890")
        lon_str, lat_str = custom_id.split('_')
        lon = float(lon_str)
        lat = float(lat_str)

        # Create polygon directly in WGS84 (approximate 2km at these latitudes)
        # Adjust cell_size_degrees based on your latitude if needed
        geometry = ee.Geometry.Polygon([[
            [lon, lat],                                    # SW corner
            [lon + cell_size_degrees, lat],                # SE corner
            [lon + cell_size_degrees, lat + cell_size_degrees],  # NE corner
            [lon, lat + cell_size_degrees]                 # NW corner
        ]])

        return geometry

    except Exception as e:
        print(f"Error creating geometry for {custom_id}: {str(e)}")
        return None
def export_image_by_custom_id(custom_id, mosaic_image, folder="Germany_Tiles_Missing"):
    """Export a single image based on custom_id"""
    try:
        # Create geometry from custom_id
        geometry = create_geometry_from_custom_id_simple(custom_id)

        if geometry is None:
            return False, f"Failed to create geometry for {custom_id}"

        # Check if geometry is valid
        area = geometry.area(maxError=1).getInfo()
        if area <= 0:
            return False, f"Invalid geometry area ({area}) for {custom_id}"

        # Export parameters (from your existing code)
        viz_params = {
            'bands': ['B4', 'B3', 'B2'],
            'min': 0.0,
            'max': 2000,
        }

        # Create export task
        task = ee.batch.Export.image.toDrive(
            image=mosaic_image.clip(geometry).visualize(**viz_params),
            description=f"missing_tile_{custom_id}",
            folder=folder,
            fileNamePrefix=f"tile_{custom_id}",
            region=geometry,
            crs='EPSG:32632',
            scale=10,
            maxPixels=1e9,
            fileFormat='GEO_TIFF'
        )

        # Start the task
        task.start()

        return True, f"Successfully started export for {custom_id}"

    except Exception as e:
        return False, f"Error exporting {custom_id}: {str(e)}"

# Batch export function for missing images
def export_missing_images_batch(missing_custom_ids, mosaic_image, batch_size=10, wait_time=30):
    """Export missing images in batches"""

    print(f"Starting export of {len(missing_custom_ids)} missing images in batches of {batch_size}")

    # Progress tracking
    progress_data = {
        'completed': [],
        'failed': [],
        'total': len(missing_custom_ids),
        'start_time': datetime.now().isoformat()
    }

    # Process in batches
    for i in range(0, len(missing_custom_ids), batch_size):
        batch_ids = missing_custom_ids[i:i + batch_size]
        batch_num = (i // batch_size) + 1
        total_batches = (len(missing_custom_ids) + batch_size - 1) // batch_size

        print(f"\n{'='*50}")
        print(f"BATCH {batch_num}/{total_batches}")
        print(f"Processing IDs: {batch_ids}")
        print(f"{'='*50}")

        # Start exports for this batch
        batch_tasks = []
        for custom_id in batch_ids:
            success, message = export_image_by_custom_id(custom_id, mosaic_image)

            if success:
                batch_tasks.append(custom_id)
                progress_data['completed'].append(custom_id)
                print(f"  ✓ Started: {custom_id}")
            else:
                progress_data['failed'].append({'custom_id': custom_id, 'error': message})
                print(f"  ✗ Failed: {custom_id} - {message}")

        print(f"\nBatch {batch_num} summary:")
        print(f"  Started: {len(batch_tasks)}")
        print(f"  Failed: {len(batch_ids) - len(batch_tasks)}")
        print(f"  Total completed so far: {len(progress_data['completed'])}")
        print(f"  Total failed so far: {len(progress_data['failed'])}")

        # Wait between batches (except for last batch)
        if i + batch_size < len(missing_custom_ids):
            print(f"\nWaiting {wait_time} seconds before next batch...")
            time.sleep(wait_time)

    # Final summary
    print(f"\n{'='*60}")
    print("MISSING IMAGES EXPORT COMPLETED!")
    print(f"{'='*60}")
    print(f"Total missing images: {len(missing_custom_ids)}")
    print(f"Successfully started: {len(progress_data['completed'])}")
    print(f"Failed to start: {len(progress_data['failed'])}")

    if progress_data['failed']:
        print(f"\nFailed exports:")
        for failed_item in progress_data['failed']:
            print(f"  - {failed_item['custom_id']}: {failed_item['error']}")

    print(f"\nCheck your Google Drive folder 'Germany_Tiles_Missing' for the exported images")
    print(f"Also check Earth Engine Task Manager for task status")

    return progress_data




Found 22822 missing custom IDs to export
First few IDs: ['13.296_52.433', '13.149_52.438', '13.177_52.419', '13.327_52.450', '13.650_52.437']


In [ ]:
# Execute the missing images export
if missing_custom_ids:
    print("Starting missing images export...")
    print("Make sure your Earth Engine is initialized and mosaic_image is defined!")

    # Uncomment the line below to run the export
    results = export_missing_images_batch(missing_custom_ids, mosaic_image, batch_size=10)

    print("\nTo run the export, uncomment the line above and ensure:")
    print("1. Earth Engine is authenticated and initialized")
    print("2. mosaic_image is defined with your image collection")
    print("3. You have enough Earth Engine quota")

else:
    print("No missing custom IDs found. Check your CSV file structure.")


Starting missing images export...
Make sure your Earth Engine is initialized and mosaic_image is defined!
Starting export of 22822 missing images in batches of 10

BATCH 1/2283
Processing IDs: ['13.296_52.433', '13.149_52.438', '13.177_52.419', '13.327_52.450', '13.650_52.437', '13.210_52.454', '13.122_52.457', '13.646_52.402', '13.267_52.434', '13.501_52.425']
  ✓ Started: 13.296_52.433
  ✓ Started: 13.149_52.438
  ✓ Started: 13.177_52.419
  ✓ Started: 13.327_52.450
  ✓ Started: 13.650_52.437
  ✓ Started: 13.210_52.454
  ✓ Started: 13.122_52.457
  ✓ Started: 13.646_52.402
  ✓ Started: 13.267_52.434
  ✓ Started: 13.501_52.425

Batch 1 summary:
  Started: 10
  Failed: 0
  Total completed so far: 10
  Total failed so far: 0

Waiting 30 seconds before next batch...

BATCH 2/2283
Processing IDs: ['13.151_52.456', '13.527_52.388', '13.588_52.404', '13.298_52.451', '13.234_52.399', '13.208_52.436', '13.117_52.403', '13.503_52.443', '13.443_52.428', '13.644_52.384']
  ✓ Started: 13.151_52.456

KeyboardInterrupt: 

In [ ]:
# # change files to be .png
# # there's multiple ways of doing this but I felt like this is the easiest
# os.chdir('/content/drive/MyDrive/Tiles/Saarland')

# # loop through each file in the folder
# for filename in os.listdir():
#     if filename.endswith('.tif'):
#         tif_path = filename
#         png_path = filename.replace('.tif', '.png')

#         # Open the .tif image and save it as .png
#         img = Image.open(tif_path)
#         img.save(png_path, format='PNG')

#         # Delete the original .tif image
#         os.remove(tif_path)


# 4 Feature Extraction with MobileNetV2

## 3.1 Feature Extraction

In [ ]:
# Configuration
image_folder = "/content/drive/MyDrive/Tiles/Tiles_all"
img_size = (200, 200)
batch_size = 100  # Process images in batches
feature_batch_size = 1000  # For feature extraction

# Get all image filenames
filenames = sorted([f for f in os.listdir(image_folder) if f.endswith('.png') or f.endswith('.jpg')])
print(f"Found {len(filenames)} images")

# Initialize the model
base_model = MobileNetV2(weights='imagenet', include_top=False, pooling='avg')

Found 70377 images


/tmp/ipython-input-5-868554673.py:12: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(weights='imagenet', include_top=False, pooling='avg')


In [ ]:
def process_images_in_batches(filenames, batch_size):
    """Generator to process images in batches"""
    for i in range(0, len(filenames), batch_size):
        batch_filenames = filenames[i:i + batch_size]
        batch_images = []

        for fname in batch_filenames:
            img_path = os.path.join(image_folder, fname)
            img = image.load_img(img_path, target_size=img_size)
            img_array = image.img_to_array(img)
            img_array = preprocess_input(img_array)
            batch_images.append(img_array)

        yield np.array(batch_images)

# Method 1: Extract features in batches and save to disk
def extract_features_batched():
    """Extract features in batches to avoid memory issues"""
    all_features = []

    print("Extracting features in batches...")
    for batch_idx, batch_images in enumerate(tqdm(process_images_in_batches(filenames, batch_size))):
        # Extract features for this batch
        batch_features = base_model.predict(batch_images, verbose=0)
        all_features.append(batch_features)

        # Clear memory
        del batch_images
        gc.collect()

        # Optional: Save intermediate results every 10 batches
        if (batch_idx + 1) % 10 == 0:
            print(f"Processed {(batch_idx + 1) * batch_size} images")

    # Combine all features
    features = np.vstack(all_features)
    del all_features
    gc.collect()

    return features

# Method 2: Use memory mapping for very large datasets
def extract_features_with_memmap():
    """Extract features using memory mapping for extremely large datasets"""
    num_images = len(filenames)
    feature_dim = 1280  # MobileNetV2 feature dimension

    # Create a memory-mapped array for features
    features_memmap = np.memmap('features_temp.dat', dtype='float32', mode='w+',
                               shape=(num_images, feature_dim))

    print("Extracting features with memory mapping...")
    processed = 0

    for batch_images in tqdm(process_images_in_batches(filenames, batch_size)):
        batch_features = base_model.predict(batch_images, verbose=0)

        # Store in memory-mapped array
        end_idx = min(processed + len(batch_features), num_images)
        features_memmap[processed:end_idx] = batch_features

        processed += len(batch_features)

        # Clear memory
        del batch_images, batch_features
        gc.collect()

    return features_memmap

# Method 3: Incremental/Online outlier detection
def incremental_outlier_detection():
    """Process outlier detection incrementally"""
    print("Using incremental approach...")

    # First pass: Extract all features
    features = extract_features_batched()

    print(f"Feature extraction complete. Shape: {features.shape}")
    print(f"Memory usage: {features.nbytes / (1024**3):.2f} GB")

    # Apply LOF
    print("Computing Local Outlier Factor...")
    lof = LocalOutlierFactor(n_neighbors=20, novelty=False)
    outlier_scores = lof.fit_predict(features)
    outlier_factors = -lof.negative_outlier_factor_

    return outlier_scores, outlier_factors, features

# Method 4: Dimensionality reduction first
def outlier_detection_with_pca():
    """Reduce dimensionality before outlier detection"""
    from sklearn.decomposition import PCA

    print("Extracting features with PCA reduction...")
    features = extract_features_batched()

    # Reduce dimensionality
    pca = PCA(n_components=100)  # Reduce to 100 dimensions
    features_reduced = pca.fit_transform(features)

    print(f"Reduced features shape: {features_reduced.shape}")
    print(f"Explained variance ratio: {pca.explained_variance_ratio_.sum():.3f}")

    # Apply LOF on reduced features
    lof = LocalOutlierFactor(n_neighbors=20, novelty=False)
    outlier_scores = lof.fit_predict(features_reduced)
    outlier_factors = -lof.negative_outlier_factor_

    return outlier_scores, outlier_factors, features_reduced

# Method 5: Subsampling approach
def outlier_detection_subsample():
    """Use a representative subsample for initial outlier detection"""
    # Take every nth image for initial analysis
    subsample_ratio = 10  # Use every 10th image
    subsample_filenames = filenames[::subsample_ratio]

    print(f"Using subsample of {len(subsample_filenames)} images")

    # Extract features for subsample
    subsample_features = []
    for batch_images in process_images_in_batches(subsample_filenames, batch_size):
        batch_features = base_model.predict(batch_images, verbose=0)
        subsample_features.append(batch_features)
        del batch_images
        gc.collect()

    subsample_features = np.vstack(subsample_features)

    # Apply LOF on subsample
    lof = LocalOutlierFactor(n_neighbors=20, novelty=True)  # Use novelty=True for later predictions
    lof.fit(subsample_features)

    return lof, subsample_features



In [ ]:
if __name__ == "__main__":
    # Method 1: Standard batched approach
    try:
        outlier_scores, outlier_factors, features = incremental_outlier_detection()

        # Find top outliers
        top_outlier_indices = np.argsort(outlier_factors)[-50:]  # Top 50 outliers

        print("Top outliers:")
        for idx in top_outlier_indices:
            print(f"Image: {filenames[idx]}, Score: {outlier_factors[idx]:.3f}")

    except MemoryError:
        print("Still running out of memory. Trying PCA approach...")

        # PCA approach for very large datasets
        outlier_scores, outlier_factors, features_reduced = outlier_detection_with_pca()

        top_outlier_indices = np.argsort(outlier_factors)[-50:]

        print("Top outliers (PCA approach):")
        for idx in top_outlier_indices:
            print(f"Image: {filenames[idx]}, Score: {outlier_factors[idx]:.3f}")


Using incremental approach...
Extracting features in batches...


10it [02:12, 11.49s/it]

Processed 1000 images


20it [04:02, 11.04s/it]

Processed 2000 images


...

Processed 69000 images


700it [3:48:54, 25.46s/it]

Processed 70000 images


704it [3:50:27, 19.64s/it]


Feature extraction complete. Shape: (70377, 1280)
Memory usage: 0.34 GB
Computing Local Outlier Factor...
Top outliers:
Image: tile_14.862_50.907.png, Score: 1.458
Image: tile_8.628_54.651.png, Score: 1.462
Image: tile_8.659_54.633.png, Score: 1.466
Image: tile_8.547_53.518.png, Score: 1.472
Image: tile_8.547_53.518_1.png, Score: 1.472
Image: tile_12.727_54.377.png, Score: 1.473
Image: tile_8.569_54.345.png, Score: 1.479
Image: tile_14.055_51.863.png, Score: 1.479
Image: tile_12.979_54.441.png, Score: 1.483
Image: tile_8.661_54.399.png, Score: 1.483
Image: tile_12.559_53.501.png, Score: 1.484
Image: tile_8.346_54.847.png, Score: 1.485
Image: tile_14.942_51.083.png, Score: 1.487
Image: tile_8.501_54.866.png, Score: 1.492
Image: tile_7.241_53.722.png, Score: 1.494
Image: tile_8.536_54.579.png, Score: 1.496
Image: tile_14.196_51.569.png, Score: 1.499
Image: tile_8.664_54.076.png, Score: 1.499
Image: tile_6.524_50.903.png, Score: 1.500
Image: tile_9.028_50.498.png, Score: 1.504
Image: tile

In [ ]:
# Create a pandas DataFrame with filenames and outlier scores
outlier_df = pd.DataFrame({
    'filename': filenames,
    'outlier_score': outlier_factors
})

# Define the path to save the CSV file in your Google Drive
csv_output_path = '/content/drive/MyDrive/image_outlier_scores_colour.csv'

# Save the DataFrame to a CSV file in your Drive
outlier_df.to_csv(csv_output_path, index=False)

print(f"Outlier scores saved successfully to {csv_output_path}")


NameError: name 'filenames' is not defined

## 3.2 Colour Extraction

In [ ]:
# Configuration
image_folder = "/content/drive/MyDrive/Tiles/Tiles_all"
img_size = (200, 200)
batch_size = 100  # Process images in batches

# Get all image filenames
filenames = sorted([f for f in os.listdir(image_folder) if f.endswith('.png') or f.endswith('.jpg')])
print(f"Found {len(filenames)} images")


Found 70377 images


In [ ]:
def extract_color_features(image_path, feature_type='histogram'):
    """Extract color features from a single image"""
    img = cv2.imread(image_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    if feature_type == 'histogram':
        # RGB histogram features
        hist_r = cv2.calcHist([img_rgb], [0], None, [32], [0, 256])
        hist_g = cv2.calcHist([img_rgb], [1], None, [32], [0, 256])
        hist_b = cv2.calcHist([img_rgb], [2], None, [32], [0, 256])
        features = np.concatenate([hist_r.flatten(), hist_g.flatten(), hist_b.flatten()])

    elif feature_type == 'mean_std':
        # Mean and standard deviation for each channel
        features = []
        for channel in range(3):  # RGB channels
            features.extend([
                np.mean(img_rgb[:, :, channel]),
                np.std(img_rgb[:, :, channel])
            ])
        features = np.array(features)

    elif feature_type == 'dominant_colors':
        # Dominant colors using K-means
        img_reshaped = img_rgb.reshape(-1, 3)
        kmeans = KMeans(n_clusters=5, random_state=42, n_init=10)
        kmeans.fit(img_reshaped)

        # Get dominant colors and their proportions
        colors = kmeans.cluster_centers_
        labels = kmeans.labels_

        # Calculate color proportions
        proportions = np.bincount(labels) / len(labels)

        # Flatten colors and proportions into feature vector
        features = np.concatenate([colors.flatten(), proportions])

    elif feature_type == 'hsv_stats':
        # HSV color space statistics
        img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        features = []
        for channel in range(3):  # HSV channels
            features.extend([
                np.mean(img_hsv[:, :, channel]),
                np.std(img_hsv[:, :, channel]),
                np.median(img_hsv[:, :, channel])
            ])
        features = np.array(features)

    elif feature_type == 'lab_stats':
        # LAB color space statistics
        img_lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
        features = []
        for channel in range(3):  # LAB channels
            features.extend([
                np.mean(img_lab[:, :, channel]),
                np.std(img_lab[:, :, channel]),
                np.median(img_lab[:, :, channel])
            ])
        features = np.array(features)

    elif feature_type == 'comprehensive':
        # Comprehensive color features combining multiple approaches
        img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        img_lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)

        features = []

        # RGB statistics
        for channel in range(3):
            features.extend([
                np.mean(img_rgb[:, :, channel]),
                np.std(img_rgb[:, :, channel])
            ])

        # HSV statistics
        for channel in range(3):
            features.extend([
                np.mean(img_hsv[:, :, channel]),
                np.std(img_hsv[:, :, channel])
            ])

        # LAB statistics
        for channel in range(3):
            features.extend([
                np.mean(img_lab[:, :, channel]),
                np.std(img_lab[:, :, channel])
            ])

        # RGB histogram (reduced bins)
        hist_r = cv2.calcHist([img_rgb], [0], None, [16], [0, 256])
        hist_g = cv2.calcHist([img_rgb], [1], None, [16], [0, 256])
        hist_b = cv2.calcHist([img_rgb], [2], None, [16], [0, 256])
        features.extend(hist_r.flatten())
        features.extend(hist_g.flatten())
        features.extend(hist_b.flatten())

        features = np.array(features)

    return features

def process_images_in_batches(filenames, batch_size, feature_type='histogram'):
    """Generator to process images in batches for color features"""
    for i in range(0, len(filenames), batch_size):
        batch_filenames = filenames[i:i + batch_size]
        batch_features = []

        for fname in batch_filenames:
            img_path = os.path.join(image_folder, fname)
            try:
                features = extract_color_features(img_path, feature_type)
                batch_features.append(features)
            except Exception as e:
                print(f"Error processing {fname}: {e}")
                # Add zero features for failed images
                if len(batch_features) > 0:
                    batch_features.append(np.zeros_like(batch_features[0]))
                else:
                    # Use a default feature size based on feature type
                    if feature_type == 'histogram':
                        batch_features.append(np.zeros(96))  # 32*3
                    elif feature_type == 'mean_std':
                        batch_features.append(np.zeros(6))   # 2*3
                    elif feature_type == 'dominant_colors':
                        batch_features.append(np.zeros(20))  # 5*3 + 5
                    elif feature_type in ['hsv_stats', 'lab_stats']:
                        batch_features.append(np.zeros(9))   # 3*3
                    elif feature_type == 'comprehensive':
                        batch_features.append(np.zeros(66))  # 6+6+6+48

        yield np.array(batch_features)

# Method 1: Extract color features in batches and save to disk
def extract_color_features_batched(feature_type='histogram'):
    """Extract color features in batches to avoid memory issues"""
    all_features = []

    print(f"Extracting {feature_type} color features in batches...")
    for batch_idx, batch_features in enumerate(tqdm(process_images_in_batches(filenames, batch_size, feature_type))):
        all_features.append(batch_features)

        # Clear memory
        gc.collect()

        # Optional: Save intermediate results every 10 batches
        if (batch_idx + 1) % 10 == 0:
            print(f"Processed {(batch_idx + 1) * batch_size} images")

    # Combine all features
    features = np.vstack(all_features)
    del all_features
    gc.collect()

    return features

# Method 2: Use memory mapping for very large datasets
def extract_color_features_with_memmap(feature_type='histogram'):
    """Extract color features using memory mapping for extremely large datasets"""
    num_images = len(filenames)

    # Determine feature dimension based on type
    if feature_type == 'histogram':
        feature_dim = 96  # 32*3
    elif feature_type == 'mean_std':
        feature_dim = 6   # 2*3
    elif feature_type == 'dominant_colors':
        feature_dim = 20  # 5*3 + 5
    elif feature_type in ['hsv_stats', 'lab_stats']:
        feature_dim = 9   # 3*3
    elif feature_type == 'comprehensive':
        feature_dim = 66  # 6+6+6+48

    # Create a memory-mapped array for features
    features_memmap = np.memmap('color_features_temp.dat', dtype='float32', mode='w+',
                               shape=(num_images, feature_dim))

    print(f"Extracting {feature_type} color features with memory mapping...")
    processed = 0

    for batch_features in tqdm(process_images_in_batches(filenames, batch_size, feature_type)):
        # Store in memory-mapped array
        end_idx = min(processed + len(batch_features), num_images)
        features_memmap[processed:end_idx] = batch_features

        processed += len(batch_features)

        # Clear memory
        del batch_features
        gc.collect()

    return features_memmap

# Method 3: Incremental/Online color outlier detection
def incremental_color_outlier_detection(feature_type='histogram'):
    """Process color outlier detection incrementally"""
    print(f"Using incremental approach with {feature_type} features...")

    # First pass: Extract all color features
    features = extract_color_features_batched(feature_type)

    print(f"Color feature extraction complete. Shape: {features.shape}")
    print(f"Memory usage: {features.nbytes / (1024**3):.2f} GB")

    # Normalize features for better outlier detection
    from sklearn.preprocessing import StandardScaler
    scaler = StandardScaler()
    features_normalized = scaler.fit_transform(features)

    # Apply LOF
    print("Computing Local Outlier Factor on color features...")
    lof = LocalOutlierFactor(n_neighbors=20, novelty=False)
    outlier_scores = lof.fit_predict(features_normalized)
    outlier_factors = -lof.negative_outlier_factor_

    return outlier_scores, outlier_factors, features_normalized

# Method 4: Dimensionality reduction first (for comprehensive features)
def color_outlier_detection_with_pca(feature_type='comprehensive'):
    """Reduce dimensionality before outlier detection"""
    from sklearn.decomposition import PCA

    print(f"Extracting {feature_type} color features with PCA reduction...")
    features = extract_color_features_batched(feature_type)

    # Normalize first
    from sklearn.preprocessing import StandardScaler
    scaler = StandardScaler()
    features_normalized = scaler.fit_transform(features)

    # Reduce dimensionality
    n_components = min(20, features_normalized.shape[1])  # Reduce to 20 dimensions or less
    pca = PCA(n_components=n_components)
    features_reduced = pca.fit_transform(features_normalized)

    print(f"Reduced features shape: {features_reduced.shape}")
    print(f"Explained variance ratio: {pca.explained_variance_ratio_.sum():.3f}")

    # Apply LOF on reduced features
    lof = LocalOutlierFactor(n_neighbors=20, novelty=False)
    outlier_scores = lof.fit_predict(features_reduced)
    outlier_factors = -lof.negative_outlier_factor_

    return outlier_scores, outlier_factors, features_reduced

# Method 5: Subsampling approach
def color_outlier_detection_subsample(feature_type='histogram'):
    """Use a representative subsample for initial outlier detection"""
    # Take every nth image for initial analysis
    subsample_ratio = 10  # Use every 10th image
    subsample_filenames = filenames[::subsample_ratio]

    print(f"Using subsample of {len(subsample_filenames)} images for {feature_type} features")

    # Extract color features for subsample
    subsample_features = []
    for batch_features in process_images_in_batches(subsample_filenames, batch_size, feature_type):
        subsample_features.append(batch_features)
        del batch_features
        gc.collect()

    subsample_features = np.vstack(subsample_features)

    # Normalize features
    from sklearn.preprocessing import StandardScaler
    scaler = StandardScaler()
    subsample_features_normalized = scaler.fit_transform(subsample_features)

    # Apply LOF on subsample
    lof = LocalOutlierFactor(n_neighbors=20, novelty=True)  # Use novelty=True for later predictions
    lof.fit(subsample_features_normalized)

    return lof, subsample_features_normalized


Using incremental approach with histogram features...
Extracting histogram color features in batches...


10it [06:08, 37.40s/it]

Processed 1000 images


20it [10:44, 26.01s/it]

Processed 2000 images


...

360it [2:43:29, 25.51s/it]

Processed 36000 images


370it [2:47:46, 25.97s/it]

Processed 37000 images


379it [2:51:48, 27.92s/it]

In [ ]:
if __name__ == "__main__":
    # Available feature types:
    # 'histogram' - RGB histograms (96 features)
    # 'mean_std' - Mean and std for RGB channels (6 features)
    # 'dominant_colors' - K-means dominant colors (20 features)
    # 'hsv_stats' - HSV statistics (9 features)
    # 'lab_stats' - LAB statistics (9 features)
    # 'comprehensive' - Combined features (66 features)

    feature_type = 'histogram'  # Change this to experiment with different color features

    # Method 1: Standard batched approach (recommended for most cases)
    try:
        outlier_scores, outlier_factors, features = incremental_color_outlier_detection(feature_type)

        # Find top outliers
        top_outlier_indices = np.argsort(outlier_factors)[-50:]  # Top 50 outliers

        print(f"\nTop color outliers using {feature_type} features:")
        for idx in top_outlier_indices:
            print(f"Image: {filenames[idx]}, Score: {outlier_factors[idx]:.3f}")

    except MemoryError:
        print("Still running out of memory. Trying PCA approach...")

        # Method 4: PCA approach for very large datasets
        outlier_scores, outlier_factors, features_reduced = color_outlier_detection_with_pca('comprehensive')

        top_outlier_indices = np.argsort(outlier_factors)[-50:]

        print(f"\nTop color outliers using comprehensive features with PCA:")
        for idx in top_outlier_indices:
            print(f"Image: {filenames[idx]}, Score: {outlier_factors[idx]:.3f}")

# Additional color-specific analysis functions
def analyze_color_outliers(outlier_indices, top_n=10):
    """Analyze what makes the color outliers unique"""
    print(f"\nAnalyzing top {top_n} color outliers:")

    for i, idx in enumerate(outlier_indices[-top_n:]):
        img_path = os.path.join(image_folder, filenames[idx])
        img = cv2.imread(img_path)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        # Basic color statistics
        mean_rgb = np.mean(img_rgb, axis=(0, 1))
        std_rgb = np.std(img_rgb, axis=(0, 1))

        print(f"{i+1}. {filenames[idx]}")
        print(f"   Mean RGB: [{mean_rgb[0]:.1f}, {mean_rgb[1]:.1f}, {mean_rgb[2]:.1f}]")
        print(f"   Std RGB:  [{std_rgb[0]:.1f}, {std_rgb[1]:.1f}, {std_rgb[2]:.1f}]")

        # Dominant color
        img_reshaped = img_rgb.reshape(-1, 3)
        from sklearn.cluster import KMeans
        kmeans = KMeans(n_clusters=1, random_state=42, n_init=10)
        dominant_color = kmeans.fit(img_reshaped).cluster_centers_[0]
        print(f"   Dominant color: [{dominant_color[0]:.1f}, {dominant_color[1]:.1f}, {dominant_color[2]:.1f}]")
        print()

def compare_feature_types():
    """Compare different color feature types on a subset"""
    print("Comparing different color feature types...")

    # Use a smaller subset for comparison
    subset_size = 1000
    subset_filenames = filenames[:subset_size]

    feature_types = ['histogram', 'mean_std', 'dominant_colors', 'hsv_stats', 'lab_stats']

    for feat_type in feature_types:
        print(f"\nTesting {feat_type} features...")

        # Extract features for subset
        features = []
        for batch_features in process_images_in_batches(subset_filenames, batch_size, feat_type):
            features.append(batch_features)

        features = np.vstack(features)

        # Normalize and apply LOF
        from sklearn.preprocessing import StandardScaler
        scaler = StandardScaler()
        features_normalized = scaler.fit_transform(features)

        lof = LocalOutlierFactor(n_neighbors=20, novelty=False)
        outlier_scores = lof.fit_predict(features_normalized)
        outlier_factors = -lof.negative_outlier_factor_

        # Show top 5 outliers for this feature type
        top_indices = np.argsort(outlier_factors)[-5:]
        print(f"Top 5 outliers for {feat_type}:")
        for idx in top_indices:
            print(f"  {subset_filenames[idx]}: {outlier_factors[idx]:.3f}")

# Memory optimization tips specific to color features
print("\nColor-based outlier detection tips:")
print("1. 'histogram' features work well for general color distribution outliers")
print("2. 'mean_std' features are fast and good for unusual brightness/contrast")
print("3. 'dominant_colors' features catch images with unusual color palettes")
print("4. 'hsv_stats' features are good for unusual saturation/hue")
print("5. 'lab_stats' features are perceptually uniform")
print("6. 'comprehensive' features combine multiple approaches but use more memory")
print("7. Always normalize color features before applying LOF")
print("8. Consider the lighting conditions in your images when choosing features")
print(f"Outlier scores saved successfully to {csv_output_path}")

## 3.3 Merging datasets

In [ ]:

# Specify the paths to your Google Sheet files in your Drive
sheet1_path = '/content/drive/MyDrive/image_outlier_scores_colour.csv'
sheet2_path = '/content/drive/MyDrive/image_outlier_scores_features.csv'

# Read the two CSV files into pandas DataFrames
try:
    df1 = pd.read_csv(sheet1_path)
    df2 = pd.read_csv(sheet2_path)
    print(f"Successfully loaded {sheet1_path} ({len(df1)} rows)")
    print(f"Successfully loaded {sheet2_path} ({len(df2)} rows)")
except FileNotFoundError as e:
    print(f"Error loading file: {e}. Please check the file paths in your Google Drive.")
except Exception as e:
    print(f"An error occurred while reading the CSV files: {e}")
    # Exit or handle the error as appropriate for your workflow
    # For now, we'll just print the error and continue with empty dataframes
    df1 = pd.DataFrame()
    df2 = pd.DataFrame()


merge_key = 'filename' # Change this if your key column has a different name

# Check if the merge key column exists in both DataFrames
if merge_key not in df1.columns:
    print(f"Error: Merge key '{merge_key}' not found in {sheet1_path}.")
elif merge_key not in df2.columns:
    print(f"Error: Merge key '{merge_key}' not found in {sheet2_path}.")
else:
    try:
        merged_df = pd.merge(df1, df2, on=merge_key, how='outer')

        print(f"\nSuccessfully merged DataFrames on column '{merge_key}'.")
        print(f"Merged DataFrame shape: {merged_df.shape}")

        # Display the first few rows of the merged DataFrame
        print("\nFirst 5 rows of the merged DataFrame:")
        print(merged_df.head())

        merged_output_path = '/content/drive/MyDrive/merged_sheets.csv'

        # Save the merged DataFrame to a new CSV file
        try:
            merged_df.to_csv(merged_output_path, index=False)
            print(f"\nMerged data saved successfully to {merged_output_path}")
        except Exception as e:
            print(f"Error saving merged DataFrame to CSV: {e}")

    except Exception as e:
        print(f"An error occurred during the merge operation: {e}")



Successfully loaded /content/drive/MyDrive/image_outlier_scores_colour.csv (70377 rows)
Successfully loaded /content/drive/MyDrive/image_outlier_scores_features.csv (70377 rows)

Successfully merged DataFrames on column 'filename'.
Merged DataFrame shape: (70377, 3)

First 5 rows of the merged DataFrame:
                   filename  outlier_score_x  outlier_score_y
0    tile_10.000_48.407.png         1.050677         1.050677
1  tile_10.000_48.407_1.png         1.050677         1.050677
2    tile_10.000_48.425.png         1.112551         1.112551
3  tile_10.000_48.425_1.png         1.112551         1.112551
4    tile_10.000_49.756.png         1.080978         1.080978

Merged data saved successfully to /content/drive/MyDrive/merged_sheets.csv


# 5 Old abandoned code

Originally the project started off quite small and the idea was to have a nice clean .ipnb collab file explaining everything with iamegs and plot creation.

However the project became quite large and was moved to GitHub as it is a lot nicer to link a GitHub project on a CV and not just a collab file.

Hence all the data preparation and chart creation are not done here. More information can be found on the GitHub page. Additionally, I did not clean this code a whole lot since in my opinion the output is the more important part for this specific excersise.

Below some the original ideas for feature extraction can be found (not viable for the final amount of data that was collected).

## 4. Local Outlier Factor on Structural Features

In [ ]:
# lof_features = LocalOutlierFactor(n_neighbors=20, novelty=False)
# feature_scores = -lof_features.fit_predict(features)
# feature_scores = -lof_features.negative_outlier_factor_


## 5. Local Outlier Factor on Color Histograms

In [ ]:
# color_features = []
# for fname in filenames:
#     img_path = os.path.join(image_folder, fname)
#     img = cv2.imread(img_path)
#     hist = cv2.calcHist([img], [0, 1, 2], None, [8, 8, 8],
#                         [0, 256, 0, 256, 0, 256]).flatten()
#     color_features.append(hist)

# color_features = np.array(color_features)

# lof_color = LocalOutlierFactor(n_neighbors=20, novelty=False)
# color_scores = -lof_color.fit_predict(color_features)
# color_scores = -lof_color.negative_outlier_factor_


## 6. Combine Feature and Color Scores

In [ ]:
# scaler = MinMaxScaler()
# feature_scores_scaled = scaler.fit_transform(feature_scores.reshape(-1, 1)).flatten()
# color_scores_scaled = scaler.fit_transform(color_scores.reshape(-1, 1)).flatten()

# combined_scores = 0.5 * feature_scores_scaled + 0.5 * color_scores_scaled
# sorted_indices = np.argsort(combined_scores)[::-1]


## 7. Print Top Unique Images by Combined Score

In [ ]:
# top_n = 5

# print(f"{'Rank':<5} {'Filename':<30} {'Pattern Score':<15} {'Color Score':<15} {'Combined Score':<15}")
# print("-" * 90)

# for i in range(top_n):
#     idx = sorted_indices[i]
#     print(f"{i+1:<5} {filenames[idx]:<30} {feature_scores_scaled[idx]:<15.4f} {color_scores_scaled[idx]:<15.4f} {combined_scores[idx]:<15.4f}")


Rank  Filename                       Pattern Score   Color Score     Combined Score 
------------------------------------------------------------------------------------------
1     tile_9.996_53.550.png          0.9421          0.5726          0.7573         
2     tile_6.954_49.527.png          0.6335          0.6748          0.6541         
3     tile_9.815_53.552.png          0.6387          0.6600          0.6493         
4     tile_9.785_53.552.png          0.4870          0.7930          0.6400         
5     tile_6.982_49.527.png          0.8933          0.3484          0.6208         


In [ ]:
# # Create a DataFrame
# df = pd.DataFrame({
#     'Filename': filenames,
#     'Pattern Score': feature_scores_scaled,
#     'Color Score': color_scores_scaled,
#     'Combined Score': combined_scores
# })

# # Optionally sort by combined score (descending)
# df = df.sort_values(by='Combined Score', ascending=False).reset_index(drop=True)

# csv_path = '/content/drive/MyDrive/unique_images_scores.csv'
# df.to_csv(csv_path, index=False)

# print(f"CSV saved to: {csv_path}")

# df.head()


CSV saved to: /content/drive/MyDrive/unique_images_scores.csv


,Filename,Pattern Score,Color Score,Combined Score
0,tile_9.996_53.550.png,0.942064,0.572569,0.757317
1,tile_6.954_49.527.png,0.633493,0.674806,0.654150
2,tile_9.815_53.552.png,0.638672,0.659974,0.649323
3,tile_9.785_53.552.png,0.487032,0.793015,0.640023
4,tile_6.982_49.527.png,0.893304,0.348368,0.620836
